# Parte 1: obtenemos los datos

In [1]:
import numpy as np
import deep_inv_opt as io
import deep_inv_opt.plot as iop

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.max_open_warning'] = 0  # Let the plots flow!
%matplotlib inline

In [4]:
u_train = io.tensor(np.linspace(0.5, 2.5, 8).reshape((-1, 1)))
u_train

tensor([[0.5000],
        [0.7857],
        [1.0714],
        [1.3571],
        [1.6429],
        [1.9286],
        [2.2143],
        [2.5000]], dtype=torch.float64)

ahora generamos los x correspondientes del modelo real

In [6]:
class ExamplePLP(io.ParametricLP):
    # Generate an LP from a given feature vector u and weight vector w.
    def generate(self, u, w):
        c = [[torch.cos(w + u**2 / 2)],
             [torch.sin(w + u**2 / 2)]]

        A_ub = [[-1.0,  0.0],      # x1 >= 0
                [ 0.0, -1.0],      # x2 >= 0
                [ 1.0,  0.0],      # x1 <= 2*w
                [ .5*w, w]]  # (1+w)*x1 + 2*(1+w)*x2 <= u

        b_ub = [[ 0.0],
                [ 0.0],
                [ 4/u],
                [   u]]
        
        return c, A_ub, b_ub, None, None

In [9]:
plp_true = ExamplePLP(weights=[0.8])

# Generate training targets by solve the true PLP at each u value.
x_train = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_train])
x_train

tensor([[6.3378e-06, 4.7761e-06],
        [7.7206e-06, 3.8456e-06],
        [2.9260e-05, 5.8345e-06],
        [2.9473e+00, 7.6424e-06],
        [2.4348e+00, 8.2233e-06],
        [2.0741e+00, 1.4854e-05],
        [1.8064e+00, 1.8646e+00],
        [1.6000e+00, 2.3250e+00]], dtype=torch.float64)

# Parte 2: definimos la red y la entrenamos

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [16]:
# definimos el dataset
class UDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Dataset con pares (u, x)
# u_data = [[1.0], [2.0], [3.0], [4.0]]
# x_targets = [[1.0, 1.5], [2.0, 2.5], [3.0, 3.5], [4.0, 4.5]]
dataset = UDataset(u_train, x_train)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

C:\Users\JairoEscanez\AppData\Local\Temp\ipykernel_6504\3770122034.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(data, dtype=torch.float32)
C:\Users\JairoEscanez\AppData\Local\Temp\ipykernel_6504\3770122034.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(targets, dtype=torch.float32)


In [18]:
# definimos la red (hay que revisar la forma de la red y el por qué)
class ParametricLPNet(nn.Module):
    def __init__(self):
        super(ParametricLPNet, self).__init__()
        # Entrada de dimensión 1, salida 8 (2 para c, 4 para A, 2 para b)
        self.fc = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, 8)  # c (2), A (4), b (2)
        )

    def forward(self, u):
        output = self.fc(u)
        c = output[:, 0:2]      # Vector de costes
        A = output[:, 2:6].reshape(-1, 2, 2)  # Matriz A (2x2)
        b = output[:, 6:8]      # Vector de restricciones
        return c, A, b


In [19]:
# funcion que resuelve el problema de programacion lineal (por ahora nos la creemos pero hay que revisarla)
# hay que tener en cuenta que esta funcion debe preservar el grafo de computo para poder hacer backpropagation
def smooth_lp(c, A, b):
    # Inicializar x con gradientes habilitados
    x = torch.zeros(A.shape[1], requires_grad=True)

    optimizer = torch.optim.SGD([x], lr=0.01)

    for _ in range(100):
        optimizer.zero_grad()
        constraint_penalty = torch.sum(torch.relu(A @ x - b))
        objective = torch.dot(c, x) + 100.0 * constraint_penalty
        objective.backward(retain_graph=True)  # Mantén el grafo activo
        optimizer.step()
    return x  # Sin detach()


In [20]:
# funcion de perdida
def my_loss(c, A, b, target):
    rs = smooth_lp(c, A, b)
    loss = torch.sum((rs - target) ** 2)
    return loss


In [21]:
# Crear la red neuronal
model = ParametricLPNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


# Entrenamiento
for epoch in range(100):
    for u_batch, x_batch in dataloader:
        c, A, b = model(u_batch)
        
        # Calcular la pérdida
        loss = my_loss(c[0], A[0], b[0], x_batch[0])  # Usar el target correspondiente

        # Backpropagation y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 7.246618270874023
Epoch 10, Loss: 0.11243888735771179
Epoch 20, Loss: 0.09430646151304245
Epoch 30, Loss: 7.003658294677734
Epoch 40, Loss: 6.5131049156188965
Epoch 50, Loss: 5.137110710144043
Epoch 60, Loss: 0.09901656955480576
Epoch 70, Loss: 0.09430646151304245
Epoch 80, Loss: 10.219924926757812
Epoch 90, Loss: 7.246618270874023


In [22]:
c,A,b=model(torch.tensor([[1.0]]))
smooth_lp(c[0], A[0], b[0])

tensor([-0.2972,  0.1004], requires_grad=True)

In [ ]:
en el dataloader hay alguna manera de añadirle un conjunto de validacion?

In [43]:
u_test = torch.tensor(np.linspace(0.1, 4, 16).reshape((-1, 1)), dtype=torch.float64)
u_test

tensor([[0.1000],
        [0.3600],
        [0.6200],
        [0.8800],
        [1.1400],
        [1.4000],
        [1.6600],
        [1.9200],
        [2.1800],
        [2.4400],
        [2.7000],
        [2.9600],
        [3.2200],
        [3.4800],
        [3.7400],
        [4.0000]], dtype=torch.float64)

In [44]:
x_test = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_test])
x_test

tensor([[5.5031e-06, 5.2914e-06],
        [5.8791e-06, 5.0125e-06],
        [6.2946e-06, 4.1114e-06],
        [1.5288e-05, 6.1703e-06],
        [6.3201e-05, 7.6848e-06],
        [2.8571e+00, 7.7249e-06],
        [2.4096e+00, 8.3817e-06],
        [2.0833e+00, 1.4402e-05],
        [1.8349e+00, 1.8074e+00],
        [1.6393e+00, 2.2303e+00],
        [6.9978e-05, 3.3749e+00],
        [1.2749e-05, 3.7000e+00],
        [6.8515e-06, 4.0250e+00],
        [1.1909e-05, 1.8498e-05],
        [2.2888e-04, 1.3794e-05],
        [9.9999e-01, 1.9565e-05]], dtype=torch.float64)

In [45]:
model.eval()

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [52]:
u = torch.tensor([[0.25]], dtype=torch.float64)
x = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u])
u,x

(tensor([[0.2500]], dtype=torch.float64),
 tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64))

In [58]:
c, A, b = c,A,b=model(torch.tensor([[0.25]]))
x_pred = smooth_lp(c[0], A[0], b[0])

In [60]:
x, x_pred, torch.norm(x - x_pred)

(tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64),
 tensor([-0.0847,  0.0720], requires_grad=True),
 tensor(0.1111, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>))

In [ ]:
# se parecen bastante, ahora el siguiente paso es hacer la validacion, ya lo dejo para mañana